In [10]:
from src.penalties import (
    get_duration_factor,
    get_activity_factor,
    get_time_penalty,
)
from src.exposure import compute_exposure_score
from src.windows import generate_time_windows
from src.ranker import label_windows

In [18]:
# Sample hourly AQI (replace with API later)
hourly_aqi = [
    {"hour": 6, "aqi": 170},
    {"hour": 7, "aqi": 180},
    {"hour": 8, "aqi": 200},
    {"hour": 9, "aqi": 220},
    {"hour": 10, "aqi": 240},
    {"hour": 11, "aqi": 230},
]

In [46]:
# User input
duration = 120  # minutes
activity = "walking"

duration_factor = get_duration_factor(duration)
activity_factor = get_activity_factor(activity)

windows = generate_time_windows(hourly_aqi, duration)

In [47]:
windows

[{'start_hour': 6, 'end_hour': 8, 'avg_aqi': 175.0, 'mid_hour': 6.5},
 {'start_hour': 7, 'end_hour': 9, 'avg_aqi': 190.0, 'mid_hour': 7.5},
 {'start_hour': 8, 'end_hour': 10, 'avg_aqi': 210.0, 'mid_hour': 8.5},
 {'start_hour': 9, 'end_hour': 11, 'avg_aqi': 230.0, 'mid_hour': 9.5},
 {'start_hour': 10, 'end_hour': 12, 'avg_aqi': 235.0, 'mid_hour': 10.5}]

In [48]:
for w in windows:
    time_penalty = get_time_penalty(w["mid_hour"])
    w["exposure"] = compute_exposure_score(
        w["avg_aqi"],
        duration_factor,
        activity_factor,
        time_penalty,
    )

In [49]:
windows[0]

{'start_hour': 6,
 'end_hour': 8,
 'avg_aqi': 175.0,
 'mid_hour': 6.5,
 'exposure': 367.5}

In [50]:
windows = sorted(windows, key=lambda x: x["exposure"])
windows = label_windows(windows)

In [51]:
windows

[{'start_hour': 6,
  'end_hour': 8,
  'avg_aqi': 175.0,
  'mid_hour': 6.5,
  'exposure': 367.5,
  'label': 'Best'},
 {'start_hour': 7,
  'end_hour': 9,
  'avg_aqi': 190.0,
  'mid_hour': 7.5,
  'exposure': 570.0,
  'label': 'Best'},
 {'start_hour': 8,
  'end_hour': 10,
  'avg_aqi': 210.0,
  'mid_hour': 8.5,
  'exposure': 630.0,
  'label': 'Acceptable'},
 {'start_hour': 9,
  'end_hour': 11,
  'avg_aqi': 230.0,
  'mid_hour': 9.5,
  'exposure': 690.0,
  'label': 'Avoid'},
 {'start_hour': 10,
  'end_hour': 12,
  'avg_aqi': 235.0,
  'mid_hour': 10.5,
  'exposure': 846.0,
  'label': 'Avoid'}]

In [52]:
# Show top results
for w in windows[:]:
    print(
        f"{w['start_hour']}:00–{w['end_hour']}:00 | "
        f"AQI {round(w['avg_aqi'])} | "
        f"Score {round(w['exposure'],1)} | "
        f"{w['label']}"
    )


6:00–8:00 | AQI 175 | Score 367.5 | Best
7:00–9:00 | AQI 190 | Score 570.0 | Best
8:00–10:00 | AQI 210 | Score 630.0 | Acceptable
9:00–11:00 | AQI 230 | Score 690.0 | Avoid
10:00–12:00 | AQI 235 | Score 846.0 | Avoid


In [ ]:
########################

In [1]:
hourly_aqi = [
    {"hour": 6, "aqi": 170},
    {"hour": 7, "aqi": 180},
    {"hour": 8, "aqi": 200},
    {"hour": 9, "aqi": 220},
    {"hour": 10, "aqi": 240},
    {"hour": 11, "aqi": 230},
]

In [8]:
import math

# def generate_time_windows(hourly_aqi: list, duration_minutes: int):
hours_needed = math.ceil(60 / 60)
windows = []

for i in range(len(hourly_aqi) - hours_needed + 1):
    window = hourly_aqi[i : i + hours_needed]

    avg_aqi = sum(h["aqi"] for h in window) / hours_needed
    mid_hour = sum(h["hour"] for h in window) / hours_needed

    windows.append({
        "start_hour": window[0]["hour"],
        "end_hour": window[-1]["hour"] + 1,
        "avg_aqi": avg_aqi,
        "mid_hour": mid_hour,
    })
    print(window)
    print(avg_aqi)
    print(mid_hour)

windows

[{'hour': 6, 'aqi': 170}]
170.0
6.0
[{'hour': 7, 'aqi': 180}]
180.0
7.0
[{'hour': 8, 'aqi': 200}]
200.0
8.0
[{'hour': 9, 'aqi': 220}]
220.0
9.0
[{'hour': 10, 'aqi': 240}]
240.0
10.0
[{'hour': 11, 'aqi': 230}]
230.0
11.0


[{'start_hour': 6, 'end_hour': 7, 'avg_aqi': 170.0, 'mid_hour': 6.0},
 {'start_hour': 7, 'end_hour': 8, 'avg_aqi': 180.0, 'mid_hour': 7.0},
 {'start_hour': 8, 'end_hour': 9, 'avg_aqi': 200.0, 'mid_hour': 8.0},
 {'start_hour': 9, 'end_hour': 10, 'avg_aqi': 220.0, 'mid_hour': 9.0},
 {'start_hour': 10, 'end_hour': 11, 'avg_aqi': 240.0, 'mid_hour': 10.0},
 {'start_hour': 11, 'end_hour': 12, 'avg_aqi': 230.0, 'mid_hour': 11.0}]

In [12]:
# Sample hourly AQI (replace with API later)
hourly_aqi = [
    {"hour": 6, "aqi": 170},
    {"hour": 7, "aqi": 180},
    {"hour": 8, "aqi": 200},
    {"hour": 9, "aqi": 220},
    {"hour": 10, "aqi": 240},
    {"hour": 11, "aqi": 230},
]

# User input
duration = 30  # minutes
activity = "walking"

duration_factor = get_duration_factor(duration)
activity_factor = get_activity_factor(activity)

windows = generate_time_windows(hourly_aqi, duration)

for w in windows:
    time_penalty = get_time_penalty(w["mid_hour"])
    w["exposure"] = compute_exposure_score(
        w["avg_aqi"],
        duration_factor,
        activity_factor,
        time_penalty,
    )

    w["NormalizedScore"] = min(100, w["exposure"] / 10)


windows = sorted(windows, key=lambda x: x["exposure"])
windows = label_windows(windows)

# Show top results
for w in windows[:3]:
    print(
        f"{w['start_hour']}:00–{w['end_hour']}:00 | "
        f"AQI {round(w['avg_aqi'])} | "
        f"Score {round(w['exposure'],1)} | "
        f"Normalized Score {round(w['NormalizedScore'],1)} | "
        f"{w['label']}"
    )


6:00–7:00 | AQI 170 | Score 119.0 | Normalized Score 11.9 | Best
7:00–8:00 | AQI 180 | Score 126.0 | Normalized Score 12.6 | Best
8:00–9:00 | AQI 200 | Score 200.0 | Normalized Score 20.0 | Acceptable


In [ ]:
import requests
from datetime import datetime, timezone


OPENAQ_URL = "https://api.openaq.org/v2/measurements"


def fetch_hourly_pm25(city: str = "Delhi", limit: int = 24):
    """
    Fetch last ~24 hourly PM2.5 measurements for a city.
    Returns list of dicts: [{hour, pm25}]
    """
    params = {
        "city": city,
        "parameter": "pm25",
        "limit": limit,
        "sort": "desc",
    }

    response = requests.get(OPENAQ_URL, params=params, timeout=10)
    response.raise_for_status()

    data = response.json()["results"]

    hourly = []
    for item in data:
        dt = datetime.fromisoformat(item["date"]["utc"].replace("Z", "+00:00"))
        hourly.append({
            "hour": dt.hour,
            "pm25": item["value"],
        })

    return hourly


ModuleNotFoundError: No module named 'requests'